In [4]:
# lets load the data from train.csv and test.csv files
import pandas as pd
import numpy as np

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')


C:\Users\kb\AppData\Local\Temp\ipykernel_2156\3475752379.py:5: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('train.csv')


In [2]:
train.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.4152954390025,High_spent_Small_value_payments,312.494088679437,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.944960,NaN,No,49.574949,118.280221622367,Low_spent_Large_value_payments,284.629162496072,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.209862853791,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.458074391071,Low_spent_Small_value_payments,223.451309727368,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.4201530862173,High_spent_Medium_value_payments,341.489231032222,Good


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

In [4]:
type(train)
train.describe(include='all')

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
count,100000,100000,100000,90015,100000,100000,100000,100000,84998.000000,100000.000000,...,100000,100000,100000.000000,90970,100000,100000.000000,95521,100000,98800,100000
unique,100000,12500,8,10139,1788,12501,16,18940,NaN,NaN,...,4,13178,NaN,404,3,NaN,91049,7,98792,3
top,0x1602,CUS_0xd40,January,Langep,38,#F%$D@*&8,_______,36585.12,NaN,NaN,...,Standard,1360.45,NaN,15 Years and 11 Months,Yes,NaN,__10000__,Low_spent_Small_value_payments,__-333333333333333333333333333__,Standard
freq,1,8,12500,44,2833,5572,7062,16,NaN,NaN,...,36479,24,NaN,446,52326,NaN,4305,25513,9,53174
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4194.170850,17.091280,...,NaN,NaN,32.285173,NaN,NaN,1403.118217,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3183.686167,117.404834,...,NaN,NaN,5.116875,NaN,NaN,8306.041270,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,303.645417,-1.000000,...,NaN,NaN,20.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1625.568229,3.000000,...,NaN,NaN,28.052567,NaN,NaN,30.306660,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3093.745000,6.000000,...,NaN,NaN,32.305784,NaN,NaN,69.249473,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5957.448333,7.000000,...,NaN,NaN,36.496663,NaN,NaN,161.224249,NaN,NaN,NaN,NaN


In [5]:
#for age
train['Age'] = pd.to_numeric(train['Age'], errors='coerce')
train['Age'].isnull().sum()
#train['Age'].iloc[0:10]



4939

In [6]:
 # for annaul income
train['Annual_Income'] = pd.to_numeric(train['Annual_Income'], errors='coerce')
train['Annual_Income'].isnull().sum()

6980

In [7]:
#for monthly_inhand_salary
train['Monthly_Inhand_Salary'] = pd.to_numeric(train['Monthly_Inhand_Salary'], errors='coerce')
train['Monthly_Inhand_Salary'].isnull().sum()


15002

In [8]:
numeric_attributes = ['Age', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate',  'Num_of_Loan', 'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit', 'Outstanding_Debt', 'Monthly_Balance']

for attribute in numeric_attributes:
      train[attribute] = pd.to_numeric(train[attribute], errors='coerce')
      print(attribute)
      print(train[attribute].isnull().sum())

Age
4939
Annual_Income
6980
Monthly_Inhand_Salary
15002
Num_Bank_Accounts
0
Num_Credit_Card
0
Interest_Rate
0
Num_of_Loan
4785
Delay_from_due_date
0
Num_of_Delayed_Payment
9746
Changed_Credit_Limit
2091
Outstanding_Debt
1009
Monthly_Balance
1209


In [9]:
from numpy import NAN


df = pd.DataFrame({'Animal': ['Falcon', 'Falcon1' , 'Parrot', 'Parrot1', 'Falcon', 'Parrot',
                              'Parrot', 'Parrot1', 'Falcon'],
                   'Max Speed': [380., 370., 26., 24.,380.,NAN, 26.,  NAN, NAN, ]})
df.head()
df.groupby("Animal", group_keys=True)[['Max Speed']].apply(lambda x: x.fillna(x.mode(numeric_only=True).iloc[0] if not x.mode(numeric_only=True).empty else x))

Max Speed
Animal              
Falcon  0      380.0
        4      380.0
        8      380.0
Falcon1 1      370.0
Parrot  2       26.0
        5       26.0
        6       26.0
Parrot1 3       24.0
        7       24.0

In [10]:
#train.describe()
x = train['Age'].groupby(by = train['Customer_ID'],group_keys = True)
print(x.head())


0         23.0
1         23.0
2       -500.0
3         23.0
4         23.0
         ...  
99992     24.0
99993     25.0
99994     25.0
99995     25.0
99996     25.0
Name: Age, Length: 62500, dtype: float64


In [33]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.utils.validation import check_is_fitted
import pandas as pd

class LoanType(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.ml_binarizer = MultiLabelBinarizer()

    def fit(self, X, y=None):
        assert isinstance(X, pd.DataFrame) and 'Type_of_Loan' in X.columns
        # Ensure the column is treated as a string and handle NaN values
        loan_types = X['Type_of_Loan'].fillna('Not Specified').astype(str).str.split(', ')
        # Clean up the loan types to remove unwanted phrases
        loan_types = loan_types.apply(lambda x: [item.strip() for item in x if 'and' not in item])
        self.ml_binarizer.fit(loan_types)      
        self._n_features = len(self.ml_binarizer.classes_)
        return self 

    def transform(self, X):
        # Check if the transformer has been fitted
        check_is_fitted(self, '_n_features')
        assert isinstance(X, pd.DataFrame) and 'Type_of_Loan' in X.columns
        # Ensure the column is treated as a string and handle NaN values
        loan_types = X['Type_of_Loan'].fillna('Not Specified').astype(str).str.split(', ')
        # Clean up the loan types to remove unwanted phrases
        loan_types = loan_types.apply(lambda x: [item.strip() for item in x if 'and' not in item])
        loan_type_dummies = self.ml_binarizer.transform(loan_types)
        
        # Create a DataFrame for the binary columns
        loan_type_df = pd.DataFrame(loan_type_dummies, columns=self.ml_binarizer.classes_, index=X.index)
        
        # Concatenate the new binary columns with the original DataFrame
        X = pd.concat([X, loan_type_df], axis=1)
        return X



In [15]:
train['Type_of_Loan'].info()



<class 'pandas.core.series.Series'>
RangeIndex: 100000 entries, 0 to 99999
Series name: Type_of_Loan
Non-Null Count  Dtype 
--------------  ----- 
88592 non-null  object
dtypes: object(1)
memory usage: 781.4+ KB


In [34]:
Loan = LoanType()
Loan.fit(train)
x=Loan.transform(train)
x.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Score,Auto Loan,Credit-Builder Loan,Debt Consolidation Loan,Home Equity Loan,Mortgage Loan,Not Specified,Payday Loan,Personal Loan,Student Loan
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,1,1,0,0,0,0,0,1,0
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,1,1,0,0,0,0,0,1,0
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,1,1,0,0,0,0,0,1,0
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,1,1,0,0,0,0,0,1,0
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,1,1,0,0,0,0,0,1,0


In [35]:
x.columns

Index(['ID', 'Customer_ID', 'Month', 'Name', 'Age', 'SSN', 'Occupation',
       'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts',
       'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Type_of_Loan',
       'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit',
       'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt',
       'Credit_Utilization_Ratio', 'Credit_History_Age',
       'Payment_of_Min_Amount', 'Total_EMI_per_month',
       'Amount_invested_monthly', 'Payment_Behaviour', 'Monthly_Balance',
       'Credit_Score', 'Auto Loan', 'Credit-Builder Loan',
       'Debt Consolidation Loan', 'Home Equity Loan', 'Mortgage Loan',
       'Not Specified', 'Payday Loan', 'Personal Loan', 'Student Loan'],
      dtype='object')

In [36]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 37 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

In [1]:
# lets create atransformer
from numpy import NAN
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted

class HiLoImputer(BaseEstimator, TransformerMixin):
   def __init__(self, hi, lo, columnName):
        self.hi = hi
        self.lo = lo
        self.columnName = columnName

   def fit(self, X, y=None):
      assert isinstance(X, pd.DataFrame) and len(X.columns) > 1
      assert self.columnName in X.columns

      self._n_features = X.shape[1]
      self.nulls = X[self.columnName].isnull
      return self

   def transform(self, X):
      check_is_fitted(self, '_n_features')
      assert isinstance(X, pd.DataFrame) and X.shape[1] == self._n_features

      #now lets replace values higher or lesser thanour range
      #X[self.columnName].app

      X[self.columnName] = np.where(X[self.columnName] > self.hi, NAN, X[self.columnName])
      X[self.columnName] = np.where(X[self.columnName] < self.lo, NAN, X[self.columnName])

      #now lets replace the NAN values with mode from other data
      X[self.columnName].groupby(by = X['Customer_ID'], group_keys = True).apply(lambda x: x.fillna(x.mode().iloc[0] if not x.mode(numeric_only=True).empty else x))
      return self

      

In [16]:
imputer = HiLoImputer(100,0, 'Age')
imputer.fit_transform(train)
train.head()

TypeError: Series.mode() got an unexpected keyword argument 'numeric_only'

In [41]:
# so the data has some realy high unrealeastic values in age, num_of_bank_accounts, num_of_credit_card, num_of_loan
# lets get a feel and get a right range for these values
# age hi value is 100, low value is 0
# num_of_bank_accounts hi value is 10, low value is 1
# num_of_credit_card hi value is 10, low value is 1
# num_of_loan hi value is 10, low value is 1
# annual income lo is 0
# monthly inhand salary lo is 0


# so now lets create a transformer



Age
38.0      2833
28.0      2829
31.0      2806
26.0      2792
32.0      2749
          ... 
5741.0       1
7178.0       1
5621.0       1
1908.0       1
1342.0       1
Name: count, Length: 1661, dtype: int64
Num_Bank_Accounts
6       13001
7       12823
8       12765
4       12186
5       12118
        ...  
1626        1
1470        1
887         1
211         1
697         1
Name: count, Length: 943, dtype: int64
Num_Credit_Card
5       18459
7       16615
6       16559
4       14030
3       13277
        ...  
791         1
1118        1
657         1
640         1
679         1
Name: count, Length: 1179, dtype: int64
Num_of_Loan
3.0       14386
2.0       14250
4.0       14016
0.0       10380
1.0       10083
          ...  
1444.0        1
392.0         1
841.0         1
1015.0        1
966.0         1
Name: count, Length: 399, dtype: int64
110.93450521244253
17.09128
22.47443
2.780339232263824
33.0
6.0
5.0
3.0
38.0
6
5
3.0
